# Machine Learning Crypto Trading Program
### Version 2.2 - linear_model from Scikit
Model duplication accuracy training and dictionary repeting variables for data limiting function

In [1]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime
from forex_python.converter import CurrencyRates
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from IPython.display import HTML

In [2]:
symbols = ["btc", "eth", "bnb", "sol", "xrp", "ada", "avax", "doge", "trx", "link", "dot", "matic", "ltc", "bch", "atom", "etc", "vet", "usdt"]
variable = {}
minimum_threshold = 0.985
timeout_runs = 10000

def print_colored_text(text, color='black'):
    html_code = f"<p style='color:{color}'>{text}</p>"
    display(HTML(html_code))

In [3]:
variable[f"btc_url"] = f'https://api.coinpaprika.com/v1/coins/btc-bitcoin/ohlcv/today'
variable[f"btc_file"] = f'BTC-AUD-DF1.csv'

variable[f"eth_url"] = f'https://api.coinpaprika.com/v1/coins/eth-ethereum/ohlcv/today'
variable[f"eth_file"] = f'ETH-AUD-DF1.csv'

variable[f"bnb_url"] = f'https://api.coinpaprika.com/v1/coins/bnb-binance-coin/ohlcv/today'
variable[f"bnb_file"] = f'BNB-AUD-DF1.csv'

variable[f"sol_url"] = f'https://api.coinpaprika.com/v1/coins/sol-solana/ohlcv/today'
variable[f"sol_file"] = f'SOL-AUD-DF1.csv'

variable[f"xrp_url"] = f'https://api.coinpaprika.com/v1/coins/xrp-xrp/ohlcv/today'
variable[f"xrp_file"] = f'XRP-AUD-DF1.csv'

variable[f"ada_url"] = f'https://api.coinpaprika.com/v1/coins/ada-cardano/ohlcv/today'
variable[f"ada_file"] = f'ADA-AUD-DF1.csv'

variable[f"avax_url"] = f'https://api.coinpaprika.com/v1/coins/avax-avalanche/ohlcv/today'
variable[f"avax_file"] = f'AVAX-AUD-DF1.csv'

variable[f"doge_url"] = f'https://api.coinpaprika.com/v1/coins/doge-dogecoin/ohlcv/today'
variable[f"doge_file"] = f'DOGE-AUD-DF1.csv'

variable[f"trx_url"] = f'https://api.coinpaprika.com/v1/coins/trx-tron/ohlcv/today'
variable[f"trx_file"] = f'TRX-AUD-DF1.csv'

variable[f"link_url"] = f'https://api.coinpaprika.com/v1/coins/link-chainlink/ohlcv/today'
variable[f"link_file"] = f'LINK-AUD-DF1.csv'

variable[f"dot_url"] = f'https://api.coinpaprika.com/v1/coins/dot-polkadot/ohlcv/today'
variable[f"dot_file"] = f'DOT-AUD-DF1.csv'

variable[f"matic_url"] = f'https://api.coinpaprika.com/v1/coins/matic-polygon/ohlcv/today'
variable[f"matic_file"] = f'MATIC-AUD-DF1.csv'

variable[f"ltc_url"] = f'https://api.coinpaprika.com/v1/coins/ltc-litecoin/ohlcv/today'
variable[f"ltc_file"] = f'LTC-AUD-DF1.csv'

variable[f"bch_url"] = f'https://api.coinpaprika.com/v1/coins/bch-bitcoin-cash/ohlcv/today'
variable[f"bch_file"] = f'BCH-AUD-DF1.csv'

variable[f"atom_url"] = f'https://api.coinpaprika.com/v1/coins/atom-cosmos/ohlcv/today'
variable[f"atom_file"] = f'ATOM-AUD-DF1.csv'

variable[f"etc_url"] = f'https://api.coinpaprika.com/v1/coins/etc-ethereum-classic/ohlcv/today'
variable[f"etc_file"] = f'ETC-AUD-DF1.csv'

variable[f"vet_url"] = f'https://api.coinpaprika.com/v1/coins/vet-vechain/ohlcv/today'
variable[f"vet_file"] = f'VET-AUD-DF1.csv'

variable[f"usdt_url"] = f'https://api.coinpaprika.com/v1/coins/usdt-tether/ohlcv/today'
variable[f"usdt_file"] = f'USDT-AUD-DF1.csv'

In [5]:
for symbol in symbols:
    
    print(f"-----------------------{symbol} Prediction-----------------------")
    
    timeout_runs_count = 0
    variable[f"{symbol}_score"] = 0

    url = variable[f"{symbol}_url"]
    response = requests.get(url)  

    if response.status_code == 200:
        data = response.json()
        print(data)
        if data:
            ohlc_data = data[0] 

            variable[f"{symbol}_open_usd"] = ohlc_data.get('open')
            variable[f"{symbol}_high_usd"] = ohlc_data.get('high')
            variable[f"{symbol}_low_usd"] = ohlc_data.get('low')
            variable[f"{symbol}_close_usd"] = ohlc_data.get('close')
            variable[f"{symbol}_volume_usd"] = ohlc_data.get('volume')

            c = CurrencyRates()
            variable[f"{symbol}_open_aud"] = c.convert('USD', 'AUD', variable[f"{symbol}_open_usd"])
            variable[f"{symbol}_high_aud"] = c.convert('USD', 'AUD', variable[f"{symbol}_high_usd"])
            variable[f"{symbol}_low_aud"] = c.convert('USD', 'AUD', variable[f"{symbol}_low_usd"])
            variable[f"{symbol}_close_aud"] = c.convert('USD', 'AUD', variable[f"{symbol}_close_usd"])
            variable[f"{symbol}_volume_aud"] = c.convert('USD', 'AUD', variable[f"{symbol}_volume_usd"])

    data = pd.read_csv(variable[f"{symbol}_file"])
    df = data[['Open', 'High', 'Low', 'Close', 'Volume', 'PredictiveClose']]
        
    while variable[f"{symbol}_score"] < minimum_threshold:
        timeout_runs_count = timeout_runs_count + 1
        X_train, X_test, y_train, y_test = train_test_split(df[['Open', 'High', 'Low', 'Close', 'Volume']], df['PredictiveClose'], test_size=0.25)
        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        model.score(X_test, y_test)

        today_data = {
            'Open': variable[f"{symbol}_open_aud"],
            'High': variable[f"{symbol}_high_aud"],
            'Low': variable[f"{symbol}_low_aud"],
            'Close': variable[f"{symbol}_close_aud"],
            'Volume': variable[f"{symbol}_volume_aud"]
        }
        today_df = pd.DataFrame([today_data])

        variable[f"{symbol}_prediction"] = model.predict(today_df[['Open', 'High', 'Low', 'Close', 'Volume']])
        variable[f"{symbol}_score"] = model.score(X_test, y_test)
        if timeout_runs_count > timeout_runs:
            variable[f"{symbol}_score"] = 2
        print(f'Counter: {timeout_runs_count} / {timeout_runs}', end='\r')
        
    variable[f"{symbol}_percent"] = variable[f"{symbol}_prediction"]/variable[f"{symbol}_close_aud"]*100-100
    if variable[f"{symbol}_percent"] <= 0:
        mark = "red"
    else:
        mark = "green"
    if variable[f"{symbol}_score"] == 2:
        mark = "yellow"
    print_colored_text(f'{symbol} Prediction', color=mark)
    print(model.score(X_test, y_test))
    print(variable[f"{symbol}_percent"])
    print(f'total runs: {timeout_runs_count}')

    

-----------------------btc Prediction-----------------------
[{'time_open': '2024-02-11T00:00:00Z', 'time_close': '2024-02-11T09:10:00Z', 'open': 47781.29868248822, 'high': 48472.650456776515, 'low': 47636.24499029698, 'close': 48208.132913531175, 'volume': 16946406290, 'market_cap': 946068314079}]


0.9852119139273542
[-0.00185445]
total runs: 4
-----------------------eth Prediction-----------------------
[{'time_open': '2024-02-11T00:00:00Z', 'time_close': '2024-02-11T09:07:00Z', 'open': 2502.41600942634, 'high': 2537.3353235438644, 'low': 2497.560420806112, 'close': 2528.587093063912, 'volume': 6877248376, 'market_cap': 303861769968}]


0.9488657558572166
[-0.55704205]
total runs: 10001
-----------------------bnb Prediction-----------------------
[{'time_open': '2024-02-11T00:00:00Z', 'time_close': '2024-02-11T09:18:00Z', 'open': 323.25698032483257, 'high': 324.7597669532722, 'low': 322.1597667909237, 'close': 324.3806310357787, 'volume': 459752965, 'market_cap': 48509632842}]


0.9856683600259667
[-0.52898251]
total runs: 27
-----------------------sol Prediction-----------------------
[{'time_open': '2024-02-11T00:00:00Z', 'time_close': '2024-02-11T09:18:00Z', 'open': 109.14394861518791, 'high': 109.81827777543266, 'low': 107.98117563742149, 'close': 109.3262863359027, 'volume': 1591811260, 'market_cap': 47796274613}]


0.9919526686847772
[-0.52873278]
total runs: 1
-----------------------xrp Prediction-----------------------
[{'time_open': '2024-02-11T00:00:00Z', 'time_close': '2024-02-11T09:18:00Z', 'open': 0.524327194015335, 'high': 0.5296757269386044, 'low': 0.5224486597286443, 'close': 0.5296757269386044, 'volume': 553846991, 'market_cap': 28871018499}]


...

0.988194204580881
[0.39286791]
total runs: 1
-----------------------atom Prediction-----------------------
[{'time_open': '2024-02-11T00:00:00Z', 'time_close': '2024-02-11T09:22:00Z', 'open': 9.986856776885038, 'high': 10.264833294904562, 'low': 9.971446582095986, 'close': 10.182327906116045, 'volume': 88635952, 'market_cap': 3932415016}]


0.9621768465704348
[-1.27517883]
total runs: 10001
-----------------------etc Prediction-----------------------
[{'time_open': '2024-02-11T00:00:00Z', 'time_close': '2024-02-11T09:23:00Z', 'open': 25.899290040364672, 'high': 26.33683689360654, 'low': 25.770155019584685, 'close': 26.275747827490576, 'volume': 228605891, 'market_cap': 3823022249}]


0.9492688704691107
[-2.07898621]
total runs: 10001
-----------------------vet Prediction-----------------------
[{'time_open': '2024-02-11T00:00:00Z', 'time_close': '2024-02-11T09:22:00Z', 'open': 0.02973658759955293, 'high': 0.030238326747925266, 'low': 0.029658802988226374, 'close': 0.03022899139154081, 'volume': 31115558, 'market_cap': 2198086503}]


0.9881305270101396
[-0.99874285]
total runs: 2296
-----------------------usdt Prediction-----------------------
[{'time_open': '2024-02-11T00:00:00Z', 'time_close': '2024-02-11T09:25:00Z', 'open': 1.0008682880341784, 'high': 1.0015410085053036, 'low': 0.9983393037851567, 'close': 1.0011571318715924, 'volume': 28843278339, 'market_cap': 96453235293}]


0.9508962718771962
[-0.0922041]
total runs: 10001


# File Formating Code

In [ ]:
variable[f"btc_fileP"] = f'BTC-AUD.csv'
variable[f"eth_fileP"] = f'ETH-AUD.csv'
variable[f"bnb_fileP"] = f'BNB-AUD.csv'
variable[f"sol_fileP"] = f'SOL-AUD.csv'
variable[f"xrp_fileP"] = f'XRP-AUD.csv'
variable[f"ada_fileP"] = f'ADA-AUD.csv'
variable[f"avax_fileP"] = f'AVAX-AUD.csv'
variable[f"doge_fileP"] = f'DOGE-AUD.csv'
variable[f"trx_fileP"] = f'TRX-AUD.csv'
variable[f"link_fileP"] = f'LINK-AUD.csv'
variable[f"dot_fileP"] = f'DOT-AUD.csv'
variable[f"matic_fileP"] = f'MATIC-AUD.csv'
variable[f"ltc_fileP"] = f'LTC-AUD.csv'
variable[f"bch_fileP"] = f'BCH-AUD.csv'
variable[f"atom_fileP"] = f'ATOM-AUD.csv'
variable[f"etc_fileP"] = f'ETC-AUD.csv'
variable[f"vet_fileP"] = f'VET-AUD.csv'
variable[f"usdt_fileP"] = f'USDT-AUD.csv'

In [ ]:
import pandas as pd
format_q = input("Format Data? (Y) (N) ")    
print("if error below: missing dataset")

if format_q == "Y":
    for symbol in symbols:
        if __name__ == "__main__":
            input_file = variable[f"{symbol}_fileP"]
            output_file = variable[f"{symbol}_file"]

            df = pd.read_csv(input_file)
            df = df.drop(df.columns[[0, 5]], axis=1)
            df['PredictiveClose'] = df.iloc[:, 3].shift(-1)
            df = df.iloc[:-1]
            df.to_csv(output_file, index=False)
else:
    print("formating queiry denied")